In [1]:
import torch
from pyro.infer import MCMC, NUTS, Predictive
import annotators_models as am
from importlib import reload
import pyro
import pyro.distributions as dist
import torch
import torch.nn.functional as F

In [2]:
def get_data():
    """
    :return: a tuple of annotator indices and class indices. The first term has shape
        `num_positions` whose entries take values from `0` to `num_annotators - 1`.
        The second term has shape `num_items x num_positions` whose entries take values
        from `0` to `num_classes - 1`.
    """
    # NB: the first annotator assessed each item 3 times
    positions = torch.tensor([1, 1, 1, 2, 3, 4, 5])
    annotations = torch.tensor(
        [
            [1, 1, 1, 1, 1, 1, 1],
            [3, 3, 3, 4, 3, 3, 4],
            [1, 1, 2, 2, 1, 2, 2],
            [2, 2, 2, 3, 1, 2, 1],
            [2, 2, 2, 3, 2, 2, 2],
            [2, 2, 2, 3, 3, 2, 2],
            [1, 2, 2, 2, 1, 1, 1],
            [3, 3, 3, 3, 4, 3, 3],
            [2, 2, 2, 2, 2, 2, 3],
            [2, 3, 2, 2, 2, 2, 3],
        ]
    )
    # we minus 1 because in Python, the first index is 0
    return positions - 1, annotations - 1

In [3]:
reload(am)

seed = 42
positions, annotations = get_data()
model = am.mace
data = (
    (annotations,)
    if model in [am.multinomial, am.item_difficulty]
    else (positions, annotations)
)

mcmc = MCMC(
    NUTS(model),
    warmup_steps=100,
    num_samples=100,
    num_chains=1,
)
mcmc.run(*data)

posterior_samples = mcmc.get_samples()

Sample: 100%|██████████| 200/200 [00:08, 24.79it/s, step size=4.82e-01, acc. prob=0.842]


In [4]:
predictive = Predictive(model, posterior_samples)
discrete_samples = predictive(*data)

In [ ]:
# item_class = torch.vmap(lambda x: torch.bincount(x, length=4))(
#     discrete_samples["c"].squeeze(-1)
# )

# print("Histogram of the predicted class of each item:")
# row_format = "{:>10}" * 5
# print(row_format.format("", *["c={}".format(i) for i in range(4)]))
# for i, row in enumerate(item_class):
#     print(row_format.format(f"item[{i}]", *row))